In [1]:
import numpy as np 
import string

np.random.seed(1234)

In [2]:
initial = {} # start of a phrase 
first_order = {} # second word only 
second_order = {}

In [3]:
def remove_punctuation(s): 
    return s.translate(str.maketrans('', '', string.punctuation))

In [4]:
!wget -P ./data/ -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘./data/robert_frost.txt’ already there; not retrieving.



In [5]:
def add2dict(d, k, v): 
    if k not in d: 
        d[k] = []
    d[k].append(v)


In [6]:
for line in open('./data/robert_frost.txt'): 
    tokens = remove_punctuation(line.rstrip().lower()).split()

    T = len(tokens) 
    for i in range(T): 
        t = tokens[i]
        if i == 0: 
            # measure the distribution of the first word 
            initial[t] = initial.get(t, 0) + 1
        else: 
            # prev word
            t_1 = tokens[i-1]

            # last word in the line 
            if i == T - 1: 
                # measure probability of ending the line with this word
                # add 2 word combination leading to end of the line in the dict
                add2dict(second_order, (t_1, t), 'END')

            if i == 1: 
                # measure distribution of the word being in a second place
                add2dict(first_order, t_1, t) 

            else: 
                # word is preceeded by 2 more words and sequence doesnt end with it
                # key = prev 2 words, value = current word
                t_2 = tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)


In [7]:
# normalize the distributions 
initial_total = sum(initial.values()) 
for t, c in initial.items(): 
    initial[t] = c / initial_total

In [9]:
# convert [cat, cat, cat, dog, dog, dog, dog, mouse...]
# into {cat: 0.5, dog: 0.4, mouse: 0.1}

def list2pdict(ts): 
    #turn each list of possibilities into a dictionary of probabilities
    d = {}
    n = len(ts) 
    for t in ts: 
        d[t] = d.get(t, 0.) + 1
    for t, c in d.items(): 
        d[t] = c / n
    return d

In [10]:
# ts is array of items that follow t_1, can be repeating 
for t_1, ts in first_order.items():
    # replace list with dictionary of probabilities 
    first_order[t_1] = list2pdict(ts)  

In [11]:
# here k is combination of 2 words, set
for k, ts in second_order.items(): 
    second_order[k] = list2pdict(ts)

In [12]:
def sample_word(d): 
    # random filter, the first word above this will be returned
    p0 = np.random.random() 

    # print "p0:", p0
    cumulative = 0 
    # loop through each item of the dict, t is the item, p is its probability
    for t, p in d.items(): 
        cumulative += p
        if p0 < cumulative: 
            return t 
        
    # should never get here 
    assert(False) 

In [17]:
def generate(): 
    for i in range(4): # generate 4 lines
        sentence = []

        # initial word 
        w0 = sample_word(initial) 
        sentence.append(w0) 

        # sample second word
        w1 = sample_word(first_order[w0])
        sentence.append(w1) 

        # second-order transition until END 
        while True: 
            w2 = sample_word(second_order[(w0, w1)])
            if w2 == "END": 
                break 
            
            sentence.append(w2)
            w0 = w1
            w1 = w2
        print(' '.join(sentence))


In [19]:
generate()

first theres the childrens house of makebelieve
are there they said it and besides
and this bill
my french indian esquimaux
